In [14]:
import re
import random
import pandas as pd
import hashlib

In [15]:
with open('certcards2.txt', mode='r', encoding='utf8') as f:
    all_cards = f.read()
    
card_split_pattern = r'\n\n\n\d+\n'
all_cards = re.split(card_split_pattern, all_cards)
# Use re.DOTALL to allow . to match newline characters
card_pattern = re.compile(r'(.+?)\n([\s\S]+)', re.DOTALL)
cards = [(match.group(1), match.group(2)) for cstring in all_cards if (match := re.search(card_pattern, cstring))]

# removing the cards that have no content and trimming
cards = [(subject, stripped_content) for subject, content in cards if len(stripped_content := content.strip()) > 5]

def hash_string_md5(s):
    """
    Hashes a string using MD5 and returns a truncated hash for efficiency.

    Parameters:
    - s (str): The input string to hash.

    Returns:
    - str: The truncated hexadecimal hash string.
    """
    if pd.isnull(s):
        return None  # Handle NaN values gracefully
    return hashlib.md5(s.encode('utf-8')).hexdigest()  # Truncate to first 12 characters


def remake_card_document(existing_cards: pd.DataFrame, filename: str='certcards2.txt'):
    with open(filename, mode='w', encoding='utf8') as f:
        i = 1
        for _, row in existing_cards.iterrows():
            print(i)
            f.write('\n'*6)
            f.write(str(i)+'\n')  
            f.write(row['head']+'\n')
            f.write(row['body'])
            i+=1
            # print(F"{row['head']}: {row['age']:.4f}")


existing_cards = pd.DataFrame(cards, columns=['head', 'body'])


# existing_cards['age'] = [random.random() for _ in existing_cards.index]
existing_cards['hash'] = existing_cards['body'].apply(hash_string_md5)
existing_cards

card_ages = pd.read_json('card_ages.json')

cards_to_age = pd.merge(
    left=existing_cards,
    right=card_ages[['hash', 'age']],
    left_on='hash', right_on='hash',
    how='left'
)

cards_to_age['head'] = cards_to_age['head'].str.strip()

if cards_to_age['age'].sum() > 2e6: age_factor = .5
else: age_factor = 1.05

cards_to_age['age'] = cards_to_age['age'].fillna(0)
cards_to_age['age'] = cards_to_age['age'] * age_factor
cards_to_age['age'] = cards_to_age['age'] + [random.random() for _ in cards_to_age.index]

cards_to_age.drop_duplicates(subset=['hash'], keep='first')
cards_to_age.sort_values('age').to_json('card_ages.json', indent=2)

existing_cards = cards_to_age

In [16]:
stats = existing_cards.groupby('head').agg({'age': ['sum', 'mean'], 'head': 'count'}).sort_values(('age', 'sum'))
stats.columns = stats.columns.to_flat_index()
stats.columns = ['_'.join(col).strip() for col in stats.columns]
stats

,age_sum,age_mean,head_count
head,,,
Jane Eyre,211.821947,21.182195,10
Azure VDI Project,216.095004,54.023751,4
General,229.962404,57.490601,4
Developer Mode,234.602500,58.650625,4
Azure Storage,260.997738,65.249435,4
Workera.ai,269.767050,53.953410,5
Dataverse,300.468691,60.093738,5
Dataverse Plugins,301.201777,43.028825,7
Diffusers Documentation,302.006970,60.401394,5


In [1]:
print(existing_cards['age'].sum())
existing_cards['age'].hist()

NameError: name 'existing_cards' is not defined

# Completely Random Shuffle

In [18]:
existing_cards = existing_cards.sample(frac=1)
remake_card_document(filename='certcards2.txt', existing_cards = existing_cards)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


# Age Shuffle

In [19]:
# existing_cards = existing_cards.sort_values('age', ascending=False)
# remake_card_document(filename='certcards2.txt', existing_cards=existing_cards)

In [20]:
existing_cards[['head', 'age']].head(n = 10)

,head,age
120,MS Identity Platform,66.390257
289,Azure Kubernetes Learning Path,21.199498
77,sklearn,59.118027
372,Home Network Project,13.292205
52,DNS,17.389275
278,Azure Kubernetes Learning Path,35.714372
264,Diffusers from Hugging Face,43.543966
89,Power BI,61.076046
5,PP365,56.924942
67,sklearn,65.304359


# Headers with fewest notes first

In [21]:
# frequency = existing_cards['head'].value_counts(ascending=True)

# existing_cards = pd.merge(
#     left=existing_cards,
#     right=frequency.rename('frequency'),
#     left_on='head', right_index=True,
#     how='left'
# )
# existing_cards['oldest'] = existing_cards.groupby('head')['age'].transform('max')
# existing_cards['oldest'] *= -1

# existing_cards.sort_values(['frequency', 'oldest', 'age'], ascending=True, inplace=True)

# remake_card_document(filename='certcards2.txt', existing_cards=existing_cards)

In [22]:
existing_cards['head'].value_counts()

head
Home Network Project              44
DNS                               36
Azure Kubernetes Learning Path    34
Search Engine Optimization        25
Diffusers Library                 21
sklearn                           20
Azure Functions                   19
PP365                             19
Python                            17
OData                             17
Power BI                          11
MS Identity Platform              11
Diffusers from Hugging Face       10
Kali Linux                        10
Jane Eyre                         10
AKS                                8
Dataverse Plugins                  7
Dataverse Queries                  7
Conditional Access                 7
Git                                6
Azure OpenAI                       6
Azure AI Search                    6
pandas                             6
Dataverse                          5
Workera.ai                         5
Diffusers Documentation            5
Azure Storage                    

# Focus on one header

In [23]:
# heads = existing_cards['head'].value_counts()
# heads = heads[heads > 5].index.tolist()
# one_header = random.sample(heads, 1)[0]
# one_header = 'Diffusers Library'
# existing_cards['pick_head'] = existing_cards['head'].apply(
#     lambda h: 0 if h == one_header else 1 
# )

# remake_card_document(existing_cards=existing_cards.sort_values('pick_head'))

## Header with oldest cards first

In [24]:
# existing_cards['oldest'] = existing_cards.groupby('head')['age'].transform('max')
# existing_cards.sort_values(['oldest', 'age'], ascending=False, inplace=True)
# remake_card_document(existing_cards)

In [25]:
existing_cards

,head,body,hash,age
120,MS Identity Platform,OpenID Connect versus SAML: The platform uses ...,44b237b8e558e4cc0f48efc03675f0a0,66.390257
289,Azure Kubernetes Learning Path,The bridge network is the default configuratio...,494d56dc564c4c95ffd920d626d954a7,21.199498
77,sklearn,LeaveOneOut: This cross-validation technique u...,8993daeff9e12b5218704d4db32f6645,59.118027
372,Home Network Project,Front: What is Docker Swarm and how does it di...,e9c23d755d0c0cd08842a9eeead3f3fd,13.292205
52,DNS,IPsec (Internet Protocol Security)\nA suite of...,fc9d2e34a0c98ace228efc95b474765f,17.389275
...,...,...,...,...
59,Diffusers Documentation,"On Windows, pip relies on a small launcher exe...",72502694afdcd59ba18edb412613112c,66.761562
261,Diffusers from Hugging Face,Prompt Embedding and Conditioning: transformer...,cef417a860da51d3e09370061623ae0d,45.566845
319,Search Engine Optimization,Ahrefs is a comprehensive SEO toolset used for...,3141690557f637178ec8e701a7d5a3da,27.601259
312,Search Engine Optimization,Yoast SEO is a WordPress plugin that assists i...,62442cee95f527f5db9e80ecf368575d,29.622531


In [26]:
15.6e3/641*30/100

7.301092043681747